In [1]:
# ==============================
#  Distributional Testing under Stochastic Policies with Doubly Robust Kernel Statistic
# ==============================

# Imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import time
from tqdm import tqdm
from sklearn.metrics import pairwise_distances
from sklearn.linear_model import LogisticRegression, LinearRegression
from scipy.stats import laplace, bernoulli
from sklearn.metrics import pairwise_kernels
import scipy.stats as st


In [25]:
# ==============================
# 1. Define Policy Classes
# ==============================
class GaussianPolicy:
    def __init__(self, w, scale=1.0):
        self.w = w
        self.scale = scale

    def sample_treatments(self, X):
        mean = X @ self.w
        return np.random.normal(mean, self.scale)

    def get_propensities(self, X, t):
        mean = X @ self.w
        return (1/(self.scale * np.sqrt(2*np.pi))) * np.exp(-0.5*((t-mean)/self.scale)**2)

class LaplacePolicy:
    def __init__(self, w, scale=0.5):
        self.w = w
        self.scale = scale

    def sample_treatments(self, X):
        mean = X @ self.w
        return laplace.rvs(loc=mean, scale=self.scale)

    def get_propensities(self, X, t):
        mean = X @ self.w
        return (1/(2*self.scale)) * np.exp(-np.abs(t-mean)/self.scale)


class BernoulliPolicy:
    def __init__(self, w):
        self.w = w

    def sample_treatments(self, X):
        prob = 1 / (1 + np.exp(-(X @ self.w)))
        return 2 * bernoulli.rvs(prob) - 1

    def get_propensities(self, X, t):
        prob = 1 / (1 + np.exp(-(X @ self.w)))
        t = (t + 1) // 2  # map {-1, 1} to {0, 1}
        return prob if t == 1 else 1 - prob

class EstimatedLoggingPolicy:
    def __init__(self, X, T):
        if np.issubdtype(T.dtype, np.integer) and np.unique(T).size == 2:
            self.model_type = 'binary'
            self.model = LogisticRegression()
            self.model.fit(X, ((T + 1) // 2))  # map {-1, 1} to {0, 1}
        else:
            self.model_type = 'continuous'
            self.model = LinearRegression()
            self.model.fit(X, T)

    def get_propensities(self, X, t):
        if self.model_type == 'binary':
            prob = self.model.predict_proba(X)[:, 1]
            t = (t + 1) // 2  # map {-1, 1} to {0, 1}
            return prob if t == 1 else 1 - prob
        else:
            mean = self.model.predict(X)
            return (1/np.sqrt(2*np.pi)) * np.exp(-0.5*(t-mean)**2)
        
# ==============================
# 2. Define Outcome Model
# ==============================
def outcome_model(X, T, beta, gamma):
    noise = 0.1 * np.random.randn(len(T))
    return X @ beta + gamma * T + noise


In [35]:
beta = np.array([0.1, 0.2, 0.3, 0.4, 0.5])
gamma = 1.0

w0 = np.random.randn(len(beta))
w = np.random.randn(len(beta))
w_prime = np.random.randn(len(beta))

sample_sizes = [50, 100, 200, 250, 300, 350]
num_experiments = 200

all_results = []

# Scenario i) No Treatment Effect
logging_pi = GaussianPolicy(w0)
pi = GaussianPolicy(w)
pi_prime = GaussianPolicy(w, scale=10)

In [36]:
ns = 200
X = np.random.randn(ns, len(beta))
X_prime = np.random.randn(ns, len(beta))

logging_T = logging_pi.sample_treatments(X)
logging_propensities = logging_pi.get_propensities(X, logging_T)
logging_Y = outcome_model(X, logging_T, beta, gamma)


In [37]:


# T_pi = pi.sample_treatments(X)
# T_pi_prime = pi_prime.sample_treatments(X_prime)
# Y_pi = outcome_model(X, T_pi, beta, gamma)
# Y_pi_prime = outcome_model(X_prime, T_pi_prime, beta, gamma)

In [38]:
from __future__ import division
import numpy as np
from sys import stdout
from sklearn.metrics import pairwise_kernels
import scipy.stats as st
from sklearn.metrics import pairwise_distances

# Code extracted from https://github.com/sorawitj/counterfactual-mean-embedding/

def MMD2u(K, w, m, n):
    """The MMD^2_u unbiased statistic.
    """
    # wx = 1.0/np.array(w[:m])[:,np.newaxis]
    # wy = 1.0/np.array(1.0 - w[m:])[:,np.newaxis]

    w_pi = w[:m]
    w_pi_prime = w[m:]
    K_pi = np.outer(w_pi,w_pi)*K[:m, :m]
    K_pi_prime = np.outer(w_pi_prime,w_pi_prime)*K[m:, m:]
    K_pi_pi_prime = np.outer(w_pi,w_pi_prime)*K[:m, m:]

    return 1.0 / (m * (m - 1.0)) * (K_pi.sum() - K_pi.diagonal().sum()) + \
        1.0 / (n * (n - 1.0)) * (K_pi_prime.sum() - K_pi_prime.diagonal().sum()) - \
        2.0 / (m * n) * K_pi_pi_prime.sum()


def compute_null_distribution(K, w, m, n, 
                              iterations=10000, verbose=False,
                              random_state=None, marker_interval=1000):
    """Compute the bootstrap null-distribution of MMD2u.
    """
    if type(random_state) == type(np.random.RandomState()):
        rng = random_state
    else:
        rng = np.random.RandomState(random_state)

    mmd2u_null = np.zeros(iterations)
    for i in range(iterations):
        if verbose and (i % marker_interval) == 0:
            print(i),
            stdout.flush()
        idx = rng.permutation(m+n)
        K_i = K[idx, idx[:, None]]
        w_i = w[idx]
        mmd2u_null[i] = MMD2u(K_i, w_i, m, n)

    if verbose:
        print("")

    return mmd2u_null


def kernel_two_sample_test_nonuniform(Y, w, m, n, kernel_function='rbf', iterations=500,
                           verbose=False, random_state=None, **kwargs):
    """Compute MMD^2_u, its null distribution and the p-value of the
    kernel two-sample test.
    Note that extra parameters captured by **kwargs will be passed to
    pairwise_kernels() as kernel parameters. E.g. if
    kernel_two_sample_test(..., kernel_function='rbf', gamma=0.1),
    then this will result in getting the kernel through
    kernel_function(metric='rbf', gamma=0.1).
    """

    K = pairwise_kernels(Y, metric=kernel_function, **kwargs)
    mmd2u = MMD2u(K, w, m, n)
    if verbose:
        print("MMD^2_u = %s" % mmd2u)
        print("Computing the null distribution.")

    mmd2u_null = compute_null_distribution(K, w, m, n, iterations,
                                           verbose=verbose,
                                           random_state=random_state)
    #p_value = max(1.0/iterations, (mmd2u_null > mmd2u).sum() /
    #              float(iterations))
    p_value = np.mean(mmd2u_null > mmd2u)
    
    if verbose:
        print("p-value ~= %s \t (resolution : %s)" % (p_value, 1.0/iterations))

    return mmd2u, mmd2u_null, p_value


In [39]:
m = 100
n = 100

pi_propensities = pi.get_propensities(X[:m], logging_T[:m])
pi_prime_propensities = pi_prime.get_propensities(X[m:], logging_T[m:])

estimate_logging_propensities = EstimatedLoggingPolicy(X, logging_T).get_propensities(X, logging_T)



In [40]:
w = np.concatenate([pi_propensities, pi_prime_propensities])[:, np.newaxis]/estimate_logging_propensities[:, np.newaxis]

Y = logging_Y[:,np.newaxis]

mmd2u, mmd2u_null, p_value = kernel_two_sample_test_nonuniform(Y, w, m, n, kernel_function='rbf', verbose=False)

In [41]:
p_value

0.058

In [ ]:
# def xMMD2(XY, T, w, kernel_function, **kwargs):
#     """The IPW-xKTE^2 statistic.
#     """
    
#     N = len(XY)
#     N2 = N//2
    
#     Ta = T[:N2]
#     Tb = T[N2:]
    
#     Ya = XY[:N2]
#     Yb = XY[N2:]
#     Y0a = Ya[T[:N2]==0]
#     Y1a = Ya[T[:N2]==1]
#     Y0b = Yb[T[N2:]==0]
#     Y1b = Yb[T[N2:]==1]
#     Y = np.vstack((Y0a, Y1a, Y0b, Y1b))
       
#     wa = w.squeeze()[:N2]
#     wb = w.squeeze()[N2:]
#     w0a = 1.0/np.array(1 - wa[Ta==0])
#     w0b = 1.0/np.array(1 - wb[Tb==0])
#     w1a = 1.0/np.array(wa[Ta==1])
#     w1b = 1.0/np.array(wb[Tb==1])
#     ww = np.concatenate((-w0a, w1a, -w0b, w1b))

    
#     # IPW
#     left_side = np.diag(ww[:N2])
#     right_side = np.diag(ww[N2:])
    
#     KY = pairwise_kernels(Y[:N2], Y[N2:], metric=kernel_function, **kwargs)
#     prod = left_side.T @ KY @ right_side
    
#     U = prod.mean(1)
#     return np.sqrt(len(U)) * U.mean() / U.std()


def xMMD2dr(Y, w, X, logging_T, kernel_function, **kwargs):
    
    """The DR-xKTE^2 statistic.
    """
    

    w_pi = w[:m]
    w_pi_prime = w[m:]
    # K_pi = np.outer(w_pi,w_pi)*K[:m, :m]
    # K_pi_prime = np.outer(w_pi_prime,w_pi_prime)*K[m:, m:]  
    
    # N = len(XY)
    # N2 = N//2
    
    m2 = m//2
    w_pi_split1 = w_pi[:m2]
    w_pi_split2 = w_pi[m2:]

    n2 = n//2
    w_pi_prime_split1 = w_pi_prime[:n2]
    w_pi_prime_split2 = w_pi_prime[n2:]

    # Ta = T[:N2]
    # Tb = T[N2:]
    

    T_pi = T[:m]
    T_pi_split1 = T_pi[:m2]
    T_pi_split2 = T_pi[m2:]

    T_pi_prime = T[m:]
    T_pi_prime_split1 = T_pi_prime[:n2]
    T_pi_prime_split2 = T_pi_prime[n2:]

    T_ope = np.vstack((T_pi_split1, T_pi_prime_split1, T_pi_split2, T_pi_prime_split2))

    Y_pi = Y[:m]
    Y_pi_split1 = Y_pi[:m2]
    Y_pi_split2 = Y_pi[m2:]

    Y_pi_prime = Y[m:]
    Y_pi_prime_split1 = Y_pi_prime[:n2]
    Y_pi_prime_split2 = Y_pi_prime[n2:]
    
    # Ya = XY[:N2]
    # Yb = XY[N2:]
    # Y0a = Ya[T[:N2]==0]
    # Y1a = Ya[T[:N2]==1]
    # Y0b = Yb[T[N2:]==0]
    # Y1b = Yb[T[N2:]==1]
    # Y = np.vstack((Y0a, Y1a, Y0b, Y1b))
    Y_ope = np.vstack((Y_pi_split1, Y_pi_prime_split1, Y_pi_split2, Y_pi_prime_split2))
    # m1 = len(Y0a)
    # m = m1 + len(Y0b)
    # n1 = len(Y1a)
    # n = n1 + len(Y1b)
    
    # X_split1 =  Xcov[:N2,:]
    X_pi = X[:m, :]
    X_pi_split1 = X_pi[:m2, :]
    X_pi_split2 = X_pi[m2:, :]
    X_pi_prime = X[m:, :]
    X_pi_prime_split1 = X_pi_prime[:n2, :]
    X_pi_prime_split2 = X_pi_prime[n2:, :]
    # Xa = Xcov[:N2,:]
    # Xb = Xcov[N2:,:]
    # X0a = Xa[Ta==0, :]
    # X0b = Xb[Tb==0, :]
    # X1a = Xa[Ta==1, :]
    # X1b = Xb[Tb==1, :]
    # X = np.vstack((X0a, X1a, X0b, X1b))
    
    X_ope = np.vstack((X_pi_split1, X_pi_prime_split1, X_pi_split2, X_pi_prime_split2))

    # wa = w.squeeze()[:N2]
    # wb = w.squeeze()[N2:]
    # w0a = 1.0/np.array(1 - wa[Ta==0])
    # w0b = 1.0/np.array(1 - wb[Tb==0])
    # w1a = 1.0/np.array(wa[Ta==1])
    # w1b = 1.0/np.array(wb[Tb==1])
    # ww = np.concatenate((-w0a, w1a, -w0b, w1b))
    w_ope = np.concatenate((-w_pi_split1, w_pi_prime_split1, -w_pi_split2, w_pi_prime_split2))
    
    # sigmaKX = np.median(pairwise_distances(X[N2:, :], X[N2:, :], metric='euclidean'))**2
    sigmaKX = np.median(pairwise_distances(X_ope[(m2+n2):, :], X_ope[(m2+n2):, :], metric='euclidean'))**2

    sigmaKT = np.median(pairwise_distances(T_ope[(m2+n2):, :], T_ope[(m2+n2):, :], metric='euclidean'))**2
    KX = pairwise_kernels(X_ope, metric='rbf', gamma=1.0/sigmaKX) 
    gamma = sigmaKX
    
    mu_pi_split1 = np.linalg.solve(KX[:m2, :m2] + gamma * np.eye(m2), KX[:m2, :m2+n2])

    # mu0a = np.linalg.solve(KX[:m1, :m1] + gamma * np.eye(m1), KX[:m1, :m1+n1])
    # zeroed_mu0a = np.vstack((mu0a, np.zeros((n1, m1+n1))))
    zeroed_mu_pi_split1 = np.vstack((mu_pi_split1, np.zeros((n2, m2+n2))))

    mu1a = np.linalg.solve(KX[m1:m1+n1, m1:m1+n1] + gamma * np.eye(n1), KX[m1:m1+n1, :m1+n1])
    zeroed_mu1a = np.vstack(( np.zeros((m1, m1+n1)), mu1a ))    
    muAa = np.hstack((zeroed_mu0a[:, :m1], zeroed_mu1a[:, m1:m1+n1]))
    
    mu0b = np.linalg.solve(KX[m1+n1:m+n1, m1+n1:m+n1] + gamma * np.eye(m-m1), KX[m1+n1:m+n1, m1+n1:])
    zeroed_mu0b = np.vstack((mu0b, np.zeros((n-n1,m+n-m1-n1))))
    mu1b = np.linalg.solve(KX[m+n1:, m+n1:] + gamma * np.eye(n-n1), KX[m+n1:, m1+n1:])
    zeroed_mu1b = np.vstack((np.zeros((m-m1,m+n-m1-n1)), mu1b))
    muAb = np.hstack((zeroed_mu0b[:, :m-m1], zeroed_mu1b[:, m-m1:]))
    
    
    #DR
    left_side = zeroed_mu1a - zeroed_mu0a + ww[:N2]*(np.eye(m1+n1) - muAa)
    right_side = zeroed_mu1b - zeroed_mu0b + ww[N2:]*(np.eye(m+n-m1-n1) - muAb)
    
    
    KY = pairwise_kernels(Y[:N2], Y[N2:], metric=kernel_function, **kwargs)
    prod = left_side.T @ KY @ right_side
    
    U = prod.mean(1)
    return np.sqrt(len(U)) * U.mean() / U.std()




# def kernel_two_sample_test_agnostic(Y, T, w, kernel_function='rbf', p=0.5,
#                            verbose=False, random_state=None, **kwargs):
#     """Compute the statistic IPW-xKTE and its p-value given normal distribution.
#     """
#     xmmd2 = xMMD2(Y, T, w, kernel_function, **kwargs)
#     if verbose:
#         print("xMMD^2 = %s" % xmmd2)
#     p_value = 1 - st.norm.cdf(xmmd2)
#     return xmmd2, p_value


def kernel_dr_two_sample_test_agnostic(Y, Xcov, T, w, kernel_function='rbf', p=0.5,
                           verbose=False, random_state=None, **kwargs): 
    """Compute the statistic AIPW-xKTE and its p-value given normal distribution.
    """ 
    xmmd2dr = xMMD2dr(Y, w, Xcov, T, kernel_function, **kwargs)
    if verbose:
        print("DR xMMD^2 = %s" % xmmd2dr)
    p_value = 1 - st.norm.cdf(xmmd2dr)
    return xmmd2dr, p_value

In [ ]:
Y = Y[:,np.newaxis]
YY0 = Y[T==0]
YY1 = Y[T==1]
YY0 = YY0[:,np.newaxis]
YY1 = YY1[:,np.newaxis]

# Gaussian RBF kernel
sigma2 = np.median(pairwise_distances(YY0, YY1, metric='euclidean'))**2
    
if method == 'DR-xKTE':
    t0 = time.time()
    value, p_value = kernel_dr_two_sample_test_agnostic(Y, X, T, w,
                                                        kernel_function='rbf',
                                                        gamma=1.0/sigma2,
                                                        verbose=False)
    times[n] = time.time() - t0
    p_values[n] = p_value
    values[n] = value